# Fine Tuning do Banco de Dados da Amazon

Faremos um fine tuning para retornar a descrição de um produto com base no banco de dados da amazon

Inicialmente vamos Conectar o notebook com o Google Drive, pois como o arquivo e muito grande, temos que te-lo salvo em algum lugar de fácilo acesso.

In [1]:
#Conexão com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Agora vamos instalar as dependencias necessárias

In [2]:
# Vamos atualizar inicialmente o pip para podermos instalar as dependecias com segurança
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!pip install --upgrade --no-deps 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-jx19bnpj/unsloth_c917deb72af0436a91c4c84f578f463b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-jx19bnpj/unsloth_c917deb72af0436a91c4c84f578f463b
  Resolved https://github.com/unslothai/unsloth.git to commit 8c155a2c07fc6521c869e44fb5f43b93fc0b81ff
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.10.1-py3-none-any.whl size=317784 sha256=1d83acd7c25be9a2f4a5a1f44212f85a6a5f33cacf3d292a06816c3b5ea7ba4b
  Stored in directory: /tmp/pip-ephem-wheel-cache-9wacohmz/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth


In [4]:
!pip install --no-deps torch xformers trl peft accelerate bitsandbytes triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 105.0 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.6/564.6 kB 28.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 115.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [bitsandbytes]


In [5]:
!pip install unsloth_zoo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 12.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 119.3 MB/s  0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: trl
    Found existing installation: trl 0.23.1
    Uninstalling trl-0.23.1:
      Successfully uninstalled trl-0.23.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [unsloth_zoo]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu

## Parte 1 - Preparação do dataset

Agora vamos preparar nossa base de dados, vamos pegar o arquivo do database e trata-lo e transformar em um json pronto para ser lido pelo Python

In [6]:
import json

json_path = "/content/drive/MyDrive/FIAP/IA/fine_tuning/trn.json"

def get_inicial_data(file_path):
    # Iremos iniciar abrindo duas novas coleção no python, uma para os titulos e outra para as descrições
    title_data = []
    content_data = []
    with open(file_path, 'r') as file:
        # Agora para cada linha testamos o Json se é valido
        for line in file:
            try:
                json_object = json.loads(line)
                title = json_object["title"]
                content = json_object["content"]
                # iremos eliminar os casos duplicados
                if (title != '' and content != ''):
                    title_data.append(title)
                    content_data.append(content)
            except json.JSONDecodeError:
                # e Caso o Json seja invalido ele não será incluido
                print(f"Eliminando linha com Json invalido: {line.strip()}")
                continue

    return title_data, content_data

# salvamos essas 2 listas em variaveis python
title_list, content_list = get_inicial_data(json_path)


Eliminando linha com Json invalido: {"uid": "B000FGG62A", "title": "The Hard Stuff", "content": "Brushing off criticisms that his albums all sound the same as casually as he peels off Chuck Berry licks from his trusty Gibson, George Thorogood keeps the faith on


In [7]:
# Checaremos quantas linhas há em cada lista e se são equivalentes
print(len(title_list))
print(len(content_list))

# Vamos checar mostrando as 10 primeiras lindas de cada Json
for i in range(1,10):
    print(title_list[i])
    print(content_list[i])

533324
533324
Mog's Kittens
Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.
Girls Ballet Tutu Neon Blue
Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun!
The Prophet
In a distant, timeless place, a mysterious prophet walks the sands. At the moment of his departure, he wishes to offer the people gifts but possesses nothing. The people gather round, each asks a question of the heart, and the man's wisdom is his gift. It is Gibran's gift to us, as well, for Gibran's prophet is rivaled in his wisdom only by the founders of the world's great religions. On the most 

In [8]:
# Agora vamos salvar os dados em um outro Json, para evitar ter que tratar os dados novamente caso precisemos reiniciar o treinamento
json_titles_path = "/content/drive/MyDrive/FIAP/IA/fine_tuning/titles.json"
json_contents_path = "/content/drive/MyDrive/FIAP/IA/fine_tuning/contents.json"

with open(json_titles_path, 'w') as json_file:
    json.dump(title_list, json_file)

with open(json_contents_path, 'w') as json_file:
    json.dump(content_list, json_file)

E com a biblioteca pandas, vamos transformar essas 2 listas em um dataframe

In [9]:
import pandas as pd

with open(json_titles_path, 'r') as json_file:
    data_titles = json.load(json_file)

with open(json_contents_path, 'r') as json_file:
    data_contents = json.load(json_file)

# Agora com o pandas, vamos transformar as duas listas de Json em um dataframe
data = {'title': data_titles, 'content': data_contents}
df = pd.DataFrame(data)

display(df.head())

,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
1,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...
2,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...
3,The Prophet,"In a distant, timeless place, a mysterious pro..."
4,Rightly Dividing the Word,--This text refers to thePaperbackedition.


Para finalmente chegarmos a um dataset que pode ser utilizado para o fine-tuning

In [10]:
import datasets
from datasets import Dataset

dataset = pd.DataFrame(df)
dataset = Dataset.from_pandas(dataset)

In [11]:
dataset

Dataset({
    features: ['title', 'content'],
    num_rows: 533324
})

## Parte 2 - Carregar o foundation model

Agora iremos carregar o modelo que será a base do nosso fine-tuning atraves do unsloth

In [12]:
from unsloth import FastLanguageModel
import torch

# Carregaremos o modelo inicial atraves do unsloth, no nosso caso iremos usar o modelo Llama de 8 bilhões de parametros
checkpoint_modelo = 'unsloth/Meta-Llama-3.1-8B'

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [13]:
# Carregando o modelo e o tokenizador pré-treinados com parametros equilibrados entre tempo de computação e eficiencia
# Colocamos a sequencia maxima de tokens de 4096 para uma descrição robusta
# Dtype sendo none a detecção é automatica
# e vamos carregar em 4bits para otimizar a memória
modelo, tokenizador = FastLanguageModel.from_pretrained(
    model_name = checkpoint_modelo,
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [14]:
modelo

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((409

In [15]:
tokenizador

PreTrainedTokenizerFast(name_or_path='unsloth/meta-llama-3.1-8b-unsloth-bnb-4bit', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|finetune_right_pad_id|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True

## Parte 3 - Testando o modelo Inicial

Agora com o modelo carregado, iremos testá-lo com um prompt

In [16]:
prompt = "Please, describe Lord of the Rings"

prompt_tokenizado = tokenizador([prompt], return_tensors = "pt").to("cuda")

In [17]:
prompt_tokenizado

{'input_ids': tensor([[128000,   5618,     11,   7664,  10425,    315,    279,  47380]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [18]:
from transformers import TextStreamer

In [21]:
FastLanguageModel.for_inference(modelo)

for i in range(10):
    streamer_texto = TextStreamer(tokenizador)
    _ = modelo.generate(**prompt_tokenizado, streamer = streamer_texto, max_new_tokens = 128)

<|begin_of_text|>Please, describe Lord of the Rings
1. The story is about a hobbit called Frodo who lives in a place called the Shire. The Shire is a peaceful place with lots of beautiful fields and forests. Frodo lives in a hobbit hole with his uncle Bilbo Baggins. Bilbo is very old and he has a magic ring. This ring has a strange power. It makes people invisible. Bilbo gives the ring to Frodo and tells him to take it to the mountain called Mount Doom. Frodo must destroy the ring there.
2. Frodo and Bilbo are not the only hobbits in the Shire. There are
<|begin_of_text|>Please, describe Lord of the Rings and the Hobbit in detail.
The Lord of the Rings and the Hobbit are two of the most beloved and iconic fantasy novels of all time, written by J.R.R. Tolkien. The Lord of the Rings is a trilogy of novels, while The Hobbit is a single novel. Both stories take place in Middle-earth, a fictional world created by Tolkien, and are set thousands of years before the events of The Lord of the R

## Parte 4 - Gerando o prompt de teste

Agora iremos criar um template de prompt para termos melhores resultados - vamos utilizar a padronização do unsloth

In [22]:
prompt_instruction = 'You are an AI assistant that helps people find information. Based on the title, or name of the product, you will generate a description of the product.'

def prompt_input(input):
    return f"Please, describe {input}"


def gerar_prompt(title, content=""):
      return f"{prompt_instruction}\n\n### Input:\n{prompt_input(title)}\n\n### Response:\n{content}"


In [23]:
# Testando o prompt com a primeira linha do nosso dataset
print(gerar_prompt(dataset[0]['title'], dataset[0]['content']))

You are an AI assistant that helps people find information. Based on the title, or name of the product, you will generate a description of the product.

### Input:
Please, describe Girls Ballet Tutu Neon Pink

### Response:
High quality 3 layer ballet tutu. 12 inches in length


In [24]:
#Precisaremos de um string marcador de fim de sentença
EOS_TOKEN = tokenizador.eos_token

In [25]:
#Agora vamos usar nosso dataset para gerar os prompts para isso criaremos ums função que crie essa
def gerar_todos_os_prompts(data):
    titles = data['title']
    contents = data['content']
    textos = []
    for title, content in zip(titles, contents):
        texto = gerar_prompt(title, content) + EOS_TOKEN
        textos.append(texto)
    return  textos

## Parte 5 - Realizando o Fine-Tuning

E Finalmente com todos os ajustes realizados vamos realizar o fine-tuning

In [26]:
# utilizando a biblioteca FastLanguageModel - iremos utilizar a funçao
# get_peft_model para definir os parametros que serão treinados no fine-tuning
# economizando assim, tempo computacional
# PEFT significa Parameter-Efficient Fine-Tuning
modelo_treinado = FastLanguageModel.get_peft_model(
    modelo,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 10,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.10.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [27]:
# importaremos as bibliotecas para a realização do treinamento
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [33]:
# e vamos aplicar tudo no SFTTrainer para realizar o Fine-tuning
trainer = SFTTrainer(
    model = modelo_treinado,
    tokenizer = tokenizador,
    train_dataset = dataset,
    dataset_text_field = "texto",
    max_seq_length = 4096,
    dataset_num_proc = 2,
    packing = False,
    peft_config = modelo.peft_config,
    formatting_func = gerar_todos_os_prompts,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 500,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 10,
        output_dir = "outputs",
        report_to="none", # Add this line to disable wandb logging
    ),
)
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/533324 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 533,324 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,2.029600
2,1.974600
3,1.912200
4,1.818300
5,1.646300
6,1.905100
7,1.686200
8,1.850700
9,1.880000
10,1.848700


TrainOutput(global_step=500, training_loss=1.8524582285881042, metrics={'train_runtime': 3957.1721, 'train_samples_per_second': 1.011, 'train_steps_per_second': 0.126, 'total_flos': 5.61132858739753e+16, 'train_loss': 1.8524582285881042, 'epoch': 0.007500131252296915})

Com o modelo treinado - vamos fazer um teste com o mesmo prompt anterior

In [34]:
FastLanguageModel.for_inference(modelo)
prompt_tokenizado = tokenizador(
[gerar_prompt("Lord of the Rings")], return_tensors = "pt").to("cuda")

for i in range(10):
    streamer_texto = TextStreamer(tokenizador)
    _ = modelo_treinado.generate(**prompt_tokenizado, streamer = streamer_texto, max_new_tokens = 256)

<|begin_of_text|>You are an AI assistant that helps people find information. Based on the title, or name of the product, you will generate a description of the product.

### Input:
Please, describe Lord of the Rings

### Response:
The Lord of the Rings is one of the most widely read books in the world. It has been translated into more than 40 languages, and is still selling in its original paperback format. The Lord of the Rings has become a cultural phenomenon, inspiring a major film trilogy and a host of other merchandising and entertainment. This new edition of The Lord of the Rings is the first time that the book has been published in a single volume.<|end_of_text|>
<|begin_of_text|>You are an AI assistant that helps people find information. Based on the title, or name of the product, you will generate a description of the product.

### Input:
Please, describe Lord of the Rings

### Response:
Lord of the Rings is a 1978 album by the British progressive rock band Emerson, Lake & Pal

## Parte 6 - Salvando o modelo para uso

Agora com o modelo treinado e testado - vamos salvar como uma biblioteca unsloth, para podermos acessar por outros metodos, como por exemplo o LMStudio

In [35]:
from huggingface_hub import notebook_login

notebook_login()

In [36]:
# Agora vamos salvar o modelo localmente
modelo_treinado.save_pretrained('modelo')
tokenizador.save_pretrained('modelo')

('modelo/tokenizer_config.json',
 'modelo/special_tokens_map.json',
 'modelo/tokenizer.json')

In [39]:
# E podemos com esse mesmo modelo - publicar no hugging face
modelo_treinado.push_to_hub('llama-3.1-8B-amazon-content')
tokenizador.push_to_hub('llama-3.1-8B-amazon-content')

README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          |  557kB /  168MB            

Saved model to https://huggingface.co/llama-3.1-8B-amazon-content


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...on-content/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


In [40]:
modelo_treinado.save_pretrained_merged("modelo_merged", tokenizador, save_method = "merged_16bit",)

config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files:  75%|███████▌  | 3/4 [00:51<00:16, 16.88s/it]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [01:02<00:00, 15.51s/it]


Unsloth: Merge process complete. Saved to `/content/modelo_merged`
